# Find love
A script that uses only standard library modules to extract GeoJSON from the GNIS place name database

In [69]:
import csv

## Set the input and output data with the below variables.

In [70]:
myData = "data/NationalFile_20200101.txt"
myOutput = "data/love-and-valentine.geojson"
myStats = "data/love-and-valentine-stats.csv"

## Set the names to search.
Add space around name to search only that name

In [71]:
loveList = [" love ", " lover ", " loves ", " valentine ", " cupid ", " heart "]

In [ ]:
# Always include home
loveCentral = """{"type": "Feature",
                 "properties": {
                    "FEATURE_NAME": "❤️",
                    "FEATURE_CLASS": "Home is where the heart is.",
                    "STATE_ALPHA": "KY, Y'ALL",
                    "ELEV_IN_FT": "-9999" },
                "geometry": {"type": "Point","coordinates":[-84.5042992, 38.0387101]}}
                ]}
                """
# Start GeoJSON template
templateStart = """{
    "type": "FeatureCollection",
    "features": [
"""

In [72]:
# Assign empty dictionary for stats compute
fcount = {}

# Search and write features
with open(myData, encoding = 'utf-8') as csvFile:
    #open empty file to write output
    with open(myOutput, "w", encoding = 'utf-8', newline = "") as geoJson:
        # create reader object 
        reader = csv.reader(csvFile, delimiter = '|')
        # write beginning of GeoJSON
        geoJson.write(templateStart)
        for row in reader:
            # Search for standalone "love" or "valentine"
            for name in loveList: 
                if name in row[1].lower():
                    if row[2] in fcount:
                        fcount[row[2]] += 1
                    else:
                        fcount[row[2]] = 1
                    # Format GeoJSON feature
                    feature = """
                    {"type": "Feature",
                     "properties": {"""
                    feature += f"""
                        "FEATURE_NAME": "{row[1]}",
                        "FEATURE_CLASS": "{row[2]}",
                        "STATE_ALPHA": "{row[3]}",
                        "ELEV_IN_FT": "{row[-4]}"
                    """
                    feature += """},
                    "geometry": {"type": "Point","coordinates":"""
                    feature += f'[{float(row[-10])}, {float(row[-11])}]'
                    feature += '}},\n'
                    # Write feature
                    geoJson.write(feature)
        # Write home
        geoJson.write(loveCentral)

In [73]:
# Sort the fcount by value
flist = list()
for key, val in list(fcount.items()):
    flist.append((val, key))

flist.sort(reverse=True)

In [74]:
for key, item in flist:
    print([key,item])

[925, 'Church']
[469, 'School']
[287, 'Cemetery']
[71, 'Hospital']
[61, 'Building']
[27, 'Locale']
[18, 'Park']
[14, 'Stream']
[11, 'Airport']
[6, 'Dam']
[5, 'Lake']
[5, 'Civil']
[4, 'Reservoir']
[4, 'Post Office']
[3, 'Summit']
[3, 'Spring']
[2, 'Populated Place']
[2, 'Bridge']
[2, 'Bench']
[1, 'Ridge']
[1, 'Mine']
[1, 'Military']
[1, 'Gap']
[1, 'Crossing']
[1, 'Canal']
[1, 'Bar']


In [75]:
with open(myStats, "w", encoding = 'utf-8', newline = "") as geoStats:
    writer = csv.writer(geoStats)
    writer.writerow(["count","feature_class"])
    for key, item in flist:
        writer.writerow([key,item])